# Toy Example

The following is a toy example using Logistic regression that has been applied and tested on our dataset. In this notebook, Logistic regression calculations are explained and how the algorithm is implemented. 

Logistic regression's representative equation is like linear regression with the key difference being the value output of a binary value instead of numeric value. Logistic models are usually implemented to predict the probability of a classification or outcome of an event. Logistic regression matches our use case of predicting the binary classification if a flight is delayed or not delayed. The algorithm also benefits of being interpretable which makes it a good choice for our toy example.

The Logistic regression is represented by the following equation: 

$$
f(X) = \beta_{0} + \sum_{j=1}^{p}[X_{j}\beta_{j}]
$$

Where X represents the input and Beta represents the weight.

Logistic regression, as a matrix, has a similar base as linear regression which is represented as:

$$ 
h(x) = \theta^Tx
$$

Where x represents a vector of inputs and Theta represents a vector of weights.

We can rewrite the expression to represent multiple features using the above equations as the following:

$$
h(x) = \theta_0 + \theta_1x1 + \theta_2x2 \ldots
$$

We then apply a sigmoid function that represents the conditional probability of logistic regression between outputs of 0 and 1. 

> The sigmoid function is: 
> 
> $$
J(t) = \frac{1}{1+e^{-t}}
> $$
>
> Which when applied to the linear regression base is:
> 
> $$
h(x) = J(\theta^Tx)
> $$
Where h(x) ranges from 0 to 1 in which both probabilities will add up to 1.

The above equations can then be rewritten together as the following expression:

$$h(x) = \frac{1}{1+e^{-\theta^TX}}$$

The purpose of the Logistic regression model is to fit with the data and to minimize the cost or loss function. We want to apply the cost or loss function in order to compute the error of the model and compare the model performance when predicting classification. 

We can then apply a cost or loss function to the algorithm which can penalize the model if the model predicts an incorrect classification. The logistic loss can be calculated using:

> The equation if the correct classification is one (y=1) and the model predicts zero
> $$
-log(1-h(x))
> $$
> 
> The equation if the correct classification is zero (y=0) and the model predicts one
> $$
-log(h(x))
> $$

The following is the squared loss function of Logistic regression which contains multiple local minima and alternatives such as hinge loss. Combining the above equations to create the complete cost function equation:

$$
cost(h(x),y)=-ylog(h(x))-log(1-h(x))(1-y)
$$

When the model predicts the label to be y=1 with the probability of 1, then the cost function is -log(1)=0 and the loss fuction is equal to 0 which indicates an ideal prediction. Similarly, when the model predicts the label to be y=0 with the probability of 1, then the cost function will be -log(1-1). When the model determines an incorrect prediction of P(y^{hat} : 0)=0.999 and its probability is P(y^{hat}=0.001), then the loss function will be -log(0.001) which is a larger error. The reason that we use 0.999 and 0.001 is because -log(0) approaches infinity. Therefore, the loss function will approach infinity, when the model prediction is y=0, as the correct prediction approaches the probability of 0.

We can compute the compute the loss of the whole training set by calculating the average over the cost of whole training set's loss with a vector of n parameters:

$$
M(\theta) = \frac{1}{n}\sum_{i=1}^{n}[y^i log(h(x^i))+log(1-h(x^i))(1-y^i)]
$$

The logistic regression weights cna be selected randomly and the cost function can be analyzed to determine if the new model has improved from the last. However, this can be inefficient. As an alternative, we can iteratively take the derivative of the cost function closer to 0 each time to obtain the slope to find its minimum, while staying cautious that we are moving towards the minimum and not its maximum. There are several other algorithms that take this approach, however, we will be using Gradient Descent for our purposes. 

In Gradient Descent, the first-order derivative of the cost function provides the slope or gradient. Its step-size or learning rate is set by the user and remains constant for each step. The multiple iterations will eventually reach a minimum. 

The following equation uses gradient descent to calculate the derivatives and minimize cost:

$$
\frac{\partial M(\theta)}{\partial\theta_j} =  \frac{1}{n}\sum_{i=1}^{n}x^i_j(h(x^i)-y^i)
$$

## Toy Example Setup

In [0]:
import datetime
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pytz import timezone
from pyspark import StorageLevel
from pyspark.sql import DataFrameNaFunctions,Window
from pyspark.sql.types import IntegerType,BooleanType,DateType,StringType,TimestampType
from pyspark.sql.functions import col,isnan,isnull,when,count,lit,to_date,lpad,date_format,rpad,regexp_replace,concat,to_utc_timestamp,to_timestamp,countDistinct,unix_timestamp,row_number,when,percent_rank,year
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler, PCA, VectorSlicer, Imputer, MinMaxScaler
from pyspark.ml.linalg import Vectors,DenseMatrix
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
blob_container = "tm30container" # The name of your container created in https://portal.azure.com
storage_account = "w261tm30" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261tm30" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "tm30key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

In [0]:
index_cols = ['UNIQUE_ID',
              'FLIGHT_UTC_DATE',
              'rank'
             ]

cat_cols = ['TIME_OF_DAY',
            'MONTH',
            'DAY_OF_WEEK',
            'OP_UNIQUE_CARRIER',
            'wnd_type',
            'cig_ceil_is_qual',
            'tmp_air_is_qual',
            'slp_prs_is_qual',
            'ga1_cov','ga1_cld',
            'ga1_bs_ht_is_qual',
            'wnd_spd_is_qual',
            'ga1_cld_qual',
            'dew_pnt_is_qual',
            'ga1_cov_is_qual',
            'aa1_is_qual',
            'vis_dist_is_qual',
            'ka1_temp',
            'FLIGHT_ROUTE'
           ]

cont_cols = ['ELEVATION',
             'wnd_dir_angle',
             'wnd_spd_rate',
             'cig_ceil_ht',
             'vis_dist',
             'tmp_air',
             'dew_pnt_tmp',
             'slp_prs',
             'aa1_prd_quant_hr',
             'aa1_dp',
             'ga1_bs_ht'
            ]

pred_cols = ['DEP_DEL15']

In [0]:
def data_pull(df, time_window = 'full', date_col='FLIGHT_UTC_DATE'):
    if time_window == '2019':
        df = df.filter(year(col(date_col)) == 2019)
    elif time_window == '2018':
        df = df.filter(year(col(date_col)) == 2018)
    elif time_window == '2017':
        df = df.filter(year(col(date_col)) == 2017)
    elif time_window == '2016':
        df = df.filter(year(col(date_col)) == 2016) 
    elif time_window == '6m':
        df = df.filter(col(date_col) < "2015-07-01T00:00:00.000")  
    elif time_window == '3m':
        df = df.filter(col(date_col) < "2015-04-01T00:00:00.000")
    return df

In [0]:
def pre_pipeline(index_cols, cont_cols, cat_cols, pred_cols):
    indexer = StringIndexer(inputCols=cat_cols, outputCols=[c+"_idx" for c in cat_cols]).setHandleInvalid("keep")
    encoder = OneHotEncoder(inputCols=[c+"_idx" for c in cat_cols], outputCols= [c+"_OHE" for c in cat_cols])
    assembler_cat = VectorAssembler(inputCols= [x+"_OHE" for x in cat_cols], outputCol="cat_features")
    imputer = Imputer(inputCols=cont_cols, outputCols=cont_cols)
    assembler_lab = StringIndexer(inputCol='DEP_DEL15', outputCol="label")
    return Pipeline(stages=[indexer, encoder, assembler_cat, imputer, assembler_lab])

The function `sigmoid` is an implementation of the above sigmoid equation that we will apply to the output of the linear regression algorithm to produce a logistic regression model

In [0]:
def sigmoid(e):
    return 1/(1 + np.exp(-e))

We implemented the cost function equation as the function `Loss`

In [0]:
def Loss(dataRDD, W): 
    augmented_data = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
    #### LOSS CAME FROM OTHER NB
    loss = augmented_data.map(lambda x: x[1] * np.log(sigmoid(W.dot(x[0]))) + (1-x[1]) * np.log(sigmoid(W.dot(x[0])))).mean()
    return loss

The gradient descent equation is translated into the `GDUpdate` function that is used to compute the model gradient and update model parameters each training step

In [0]:
def GDUpdate(dataRDD, W, learningRate = 0.1):
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1])).cache()
    grad = (augmentedData.map(lambda x: 2*(sigmoid(W.dot(x[0]))-x[1])*x[0]).mean())
    new_model = W-(learningRate*grad)
    return new_model

The `GradientDescent` function encompasses all of the above functions and manages the model training process returning data from all model training steps for the train data, test data, and models.

In [0]:
def GradientDescent(trainRDD, testRDD, wInit, nSteps = 20, learningRate = 0.1, verbose = False):
    train_history, test_history, model_history = [], [], []
    model = wInit
    for idx in range(nSteps): 
        model = GDUpdate(trainRDD,model,learningRate)
        training_loss = Loss(trainRDD,model)
        test_loss = Loss(testRDD,model)
        train_history.append(training_loss)
        test_history.append(test_loss)
        model_history.append(model)
    return train_history, test_history, model_history

In [0]:
test_pq = spark.read.parquet(f"{blob_url}/2022-03-24_data_chkpt_PQ_full")

test_pq = test_pq.na.replace('', None, 'wnd_type') \
                 .na.replace('', None, 'ga1_cld') \
                 .na.replace('', None, 'ga1_cov') \
                 .withColumn('wnd_dir_angle',col('wnd_dir_angle').cast(IntegerType())) \
                 .withColumn('ka1_temp', when(isnull('ka1_temp'), '0').when(col('ka1_temp') < 0, -1).otherwise('1')) \
                 .withColumn('FLIGHT_ROUTE', concat(col('ORIGIN'),lit("-"),col('DEST')))

df_6m = data_pull(test_pq, time_window='6m', date_col='FLIGHT_UTC_DATE')

In [0]:
pre_pipeline_test = pre_pipeline(index_cols, cont_cols, cat_cols, pred_cols)

In [0]:
df_fit = pre_pipeline_test.fit(df_6m)
df_transform = df_fit.transform(df_6m)
df_select = df_transform.select('label','cat_features')

In [0]:
train,test = df_select.randomSplit([0.7,0.3],seed=2022)
trainRDD = train.sample(0.0002, 2022).rdd.map(lambda x: (x[1].toArray(),x[0])).cache()
testRDD = test.sample(0.0002, 2022).rdd.map(lambda x: (x[1].toArray(),x[0])).cache()

In [0]:
trainRDD.count()

Out[40]: 450

In [0]:
testRDD.count()

Out[41]: 189

In [0]:
wInit = np.insert(np.zeros(trainRDD.take(1)[0][0].shape[0]),0,trainRDD.map(lambda x: x[1]).mean())

In [0]:
model_results = GradientDescent(trainRDD, testRDD, wInit)

In [0]:
df_pred = pd.DataFrame(model_results).T
df_pred

Out[46]:

,0,1,2
0,-1.111493,-1.10947,"[0.1428168070880142, -0.03214797868416597, -0...."
1,-1.335307,-1.331895,"[0.11921523406226187, -0.04561971930336953, -0..."
2,-1.439474,-1.435026,"[0.10880359075296206, -0.053562985977021776, -..."
3,-1.492099,-1.486781,"[0.10359213213328998, -0.05930822881277332, -0..."
4,-1.519784,-1.513681,"[0.1008071443671205, -0.06400990456814812, -0...."
5,-1.534673,-1.527834,"[0.09924607031163524, -0.06816784298329132, -0..."
6,-1.542803,-1.535257,"[0.09832670603410014, -0.0720237597434065, -0...."
7,-1.54731,-1.539075,"[0.09775060863627648, -0.07570148267411392, -0..."
8,-1.549862,-1.540952,"[0.09736007695254821, -0.07926646177217157, -0..."
9,-1.551359,-1.541783,"[0.09707040813379739, -0.08275391772752123, -0..."


## Resources
- https://towardsdatascience.com/building-a-logistic-regression-in-python-301d27367c24
- [Week 6 Slides](https://docs.google.com/presentation/d/1VDPrWlJFgbsEIh5MhUoSbQcAKkgBve06lYhubqA48QE/edit#slide=id.gf4c8f735ab_0_200)
- [W261 Homework 4](https://github.com/UCB-w261/main/blob/main/Assignments/HW4/docker/student/hw4_Workbook.ipynb)